In [ ]:
import json
import numpy as np
import re
import random
import pandas as pd
import datetime
from datetime import timedelta
from datetime import datetime
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer


In [ ]:

tok = WordPunctTokenizer()
# Functions
def pre_process(text):
  text = re.sub(r"(?:\@|https?\://)\S+", "", text)
  text = ' '.join(x for x in text.split() if x.startswith('http') == False and x.startswith('www') == False)
  text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text = re.sub(r'^www?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  # extra step to make sure html tags are completely removed
  clean = re.compile('<.*>|<.*\"')
  text = re.sub(clean, '', text)
  text = re.sub("(pic.*)","", text)
  text = re.sub(r"http\S+","", text)
  text = re.sub(r"nyti\S+","", text)
  text = re.sub(r"lat\S+","", text)
  text = re.sub(r"bos\S+","", text)
  text = re.sub(r"wapo\S+","", text)
  text = re.sub("(twitter.*)","", text)
  text=re.sub("<!--?.*?-->","",text)
  text = re.sub("(dlvr.*)","", text)
  text=text.replace(u'\u200c'," ")

  return text
def encoding_fixer(text):

    pat1 = r'@[A-Za-z0-9]+'
    pat2 = r'https?://[A-Za-z0-9./]+'
    combined_pat = r'|'.join((pat1, pat2))

    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()
def fix_time_format(t):
  try:
    return datetime.strptime(t.strip(' \t\r\n'), "%B %d, %Y")
  except:
    try:
      return datetime.strptime(t.strip(' \t\r\n'), "%b %d, %Y")
    except:
      try:
        return datetime.strptime(t.strip(' \t\r\n'), "%d-%b-%y")
      except:
        print("error")

In [ ]:
# !pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.3.1/en_core_web_lg-2.3.1.tar.gz
import spacy
!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')

In [ ]:
def spacy_ner(document):
  return {(ent.text.strip(), ent.label_) for ent in nlp(document).ents}

def NER(dataset):
  ne_list=[]
  for i in range(dataset.shape[0]):

    spcy_ner=spacy_ner(dataset.title[i])
    ne=[]
    for j in spcy_ner:
      
      ne.append(j[0])
    ne_list.append(ne)

  dataset["NE"]=ne_list
  return dataset